In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from mask_to_submission import *
import scipy.misc
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
# functions for cross validation

def cross_validation(model, k_fold):
    seed = 1
    
    k_indices = build_k_indices(img_patches.shape[0], k_fold, seed)

    res = []
    acc = []
    for k in range(k_fold):
        score, accuracy = cross_validation_step(_X, _Y, k_indices, k, model)
        res.append(score)
        acc.append(accuracy)
    res = np.asarray(res)
    acc = np.asarray(acc)
    print('Cross validation F-score: ' + str(np.mean(res)) + ', std=' + str(np.std(res)))
    print('Cross validation accuracy: ' + str(np.mean(acc)) + ', std=' + str(np.std(acc)))
    return res,acc


def cross_validation_step(X, Y, k_indices, k, model):
    
    testing_im = X[k_indices[k]].tolist()
    testing_gt = Y[k_indices[k]].tolist()
    
    training_im = []
    training_gt = []
    for j in range(len(k_indices)):
        if j != k:
            training_im += X[k_indices[j]].tolist()
            training_gt += Y[k_indices[j]].tolist()

    model.fit(training_im, training_gt)
    prediction = model.predict(testing_im)
    
    score = f1_score(testing_gt,prediction,average = "weighted")
    accuracy = accuracy_score(testing_gt,prediction)
    print('Cross validation F-score: ' + str(score))
    print('Cross validation accuracy: ' + str(accuracy))
    
    return score, accuracy


def build_k_indices(N, k_fold, seed):
    interval = int(N/k_fold)
    np.random.seed(seed)
    
    indices = np.random.permutation(N)

    k_indices = None
    if N%k_fold == 0:
        k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    else:
        k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold-1)]
        k_indices.append(indices[(k_fold-1)*interval : N])
    return np.array(k_indices)

In [3]:
# helper functions

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

# Convert array of labels to an image

def label_to_img(imgwidth, imgheight, w, h, labels):
    im = np.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[j:j+w, i:i+h] = labels[idx]
            idx = idx + 1
    return im

foreground_threshold = 0.25

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0
    
#Extract 9-dimensional features consisting of average RGB color as well as variance
def extract_features(img):
    feat_m = np.mean(img, axis=(0,1))
    feat_me = np.median(img, axis=(0,1))
    feat_v = np.var(img, axis=(0,1))
    feat = np.append(np.append(feat_m, feat_me), feat_v)
    return feat

# Extract features for a given image
def extract_img_features(filename):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size)
    X = np.asarray([ extract_features(img_patches[i]) for i in range(len(img_patches))])
    return X

# Extract 2-dimensional features consisting of average gray color as well as variance
def extract_features_2d(img):
    feat_m = np.mean(img)
    feat_v = np.var(img)
    feat = np.append(feat_m, feat_v)
    return feat

# Extract features for a given image
def extract_img_features_2d(filename):
    img = load_image(filename)
    img_patches = img_crop(img, patch_size, patch_size)
    X = np.asarray([ extract_features_2d(img_patches[i]) for i in range(len(img_patches))])
    return X

PIXEL_DEPTH = 255
def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * PIXEL_DEPTH).round().astype(np.uint8)
    return rimg


def make_img_overlay(img, predicted_img):
    w = img.shape[0]
    h = img.shape[1]
    color_mask = np.zeros((w, h, 3), dtype=np.uint8)
    color_mask[:,:,0] = predicted_img*PIXEL_DEPTH

    img8 = img_float_to_uint8(img)
    background = Image.fromarray(img8, 'RGB').convert("RGBA")
    overlay = Image.fromarray(color_mask, 'RGB').convert("RGBA")
    new_img = Image.blend(background, overlay, 0.2)
    return new_img

In [4]:
# directories of images
root_dir = "training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"

files = os.listdir(image_dir)

In [5]:
# loading training images and groundtruth
imgs = [load_image(image_dir + files[i]) for i in range(len(files))]
gt_imgs = [load_image(gt_dir + files[i]) for i in range(len(files))]

In [6]:
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(len(imgs))]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(len(gt_imgs))]

In [7]:
# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [8]:
# Feature extraction
_X = np.asarray([extract_features(img_patches[i]) for i in range(len(img_patches))])
_Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

In [9]:
# Model creation and 4-fold cross validation

model = GradientBoostingClassifier(n_estimators=500, max_depth=10)
Z = cross_validation(model,4)

Cross validation F-score: 0.786542297185
Cross validation accuracy: 0.798208
Cross validation F-score: 0.788710700082
Cross validation accuracy: 0.800192
Cross validation F-score: 0.786548890267
Cross validation accuracy: 0.79872
Cross validation F-score: 0.78884790812
Cross validation accuracy: 0.79904
Cross validation F-score: 0.787662448913, std=0.00111791064002
Cross validation accuracy: 0.79904, std=0.000728307627311


In [10]:
# We now fit the model with all the data

model.fit(_X, _Y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=10,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=500, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [11]:
# We create groundtruth and overlayed images to analyze graphically the results

w = 608
h = 608
test_dir = './testing/groundtruth/'
test_overlay_dir = './testing/overlay/'

if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    
if not os.path.exists(test_overlay_dir):
    os.makedirs(test_overlay_dir)    

for i in range(1,51):
    image_dir = "./test_set_images/"  + "test_" + str(i) +"/"
    file = os.listdir(image_dir)[0]
    Xi = extract_img_features(image_dir+file)
    Zi = model.predict(Xi)

    predicted_im = label_to_img(w, h, patch_size, patch_size, Zi)

    img = load_image("./test_set_images/"  + "test_" + str(i) +"/test_{0}.png".format(i))
    new_img = make_img_overlay(img, predicted_im)
    
    img_file_name = test_dir + 'satImage_' + '%.3d'%i + '.png'
    img_overlay_name = test_overlay_dir + 'satImage_' + '%.3d'%i + '.png'
    scipy.misc.imsave(img_overlay_name, new_img)
    scipy.misc.imsave(img_file_name, predicted_im)

In [12]:
# Submission generation

submission_filename = 'dummy_submission.csv'
image_filenames = []
for i in range(1, 51):
    image_filename = 'testing/groundtruth/satImage_' + '%.3d' % i + '.png'
    image_filenames.append(image_filename)
masks_to_submission(submission_filename, *image_filenames)